In [24]:
import pandas as pd

import data.breathe_data as bd
import data.helpers as dh
import models.builders as mb
from plotly.subplots import make_subplots
from plotly import graph_objects as go
import inference.helpers as ih

In [2]:
# Checked that obs indices are correct, see ipynb mentioned above(01.05.2025)
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_conservative_smoothing_with_idx")

INFO:root:* Checking for same day measurements *


In [12]:
df_fev1 = bd.load_meas_from_excel(
    "infer_AR_using_fev1_01052025", ["AR", "HFEV1", "HO2Sat"], bypass_sanity_checks=True
).drop(columns=["HO2Sat"])
df_fev1.rename(
    {
        "AR": "AR_fev1",
        "HFEV1": "HFEV1_fev1",
    },
    axis=1,
    inplace=True,
)

In [ ]:
df_fev1_o2 = bd.load_meas_from_excel(
    "infer_AR_using_o2sat_fev1_01052025", ["AR", "HFEV1"], bypass_sanity_checks=True
).drop(columns=["IA", "HO2Sat"])
df_fev1_o2.rename(
    {
        "AR": "AR_fev1_o2",
        "HFEV1": "HFEV1_fev1_o2",
    },
    axis=1,
    inplace=True,
)

In [ ]:
df_fev1_o2_2d = bd.load_meas_from_excel(
    "infer_AR_using_two_days_model_o2_fev1_01052025",
    ["Airway resistance (%)", "Healthy FEV1 (L)"],
    date_cols=["Day"],
    bypass_sanity_checks=True,
).drop(columns=["Healthy O2 saturation (%)"])
df_fev1_o2_2d.rename(
    {
        "Airway resistance (%)": "AR_fev1_o2_2d",
        "Healthy FEV1 (L)": "HFEV1_fev1_o2_2d",
        "Day": "Date Recorded",
    },
    axis=1,
    inplace=True,
)

In [76]:
# join df to df_fev1 on ID and Date Recorded
cols = ["ID", "Date Recorded", "ecFEV1 % Predicted"]
df_res = pd.merge(df[cols], df_fev1, on=["ID", "Date Recorded"], how="inner")
df_res = pd.merge(df_res, df_fev1_o2, on=["ID", "Date Recorded"], how="inner")
df_res = pd.merge(df_res, df_fev1_o2_2d, on=["ID", "Date Recorded"], how="inner")

In [201]:
# Add max ecFEV1 % pred
df_max_fev = (
    df_res.groupby("ID")
    .apply(lambda df: df["ecFEV1 % Predicted"].max())
    .reset_index()
    .rename(columns={0: "max ecFEV1 % Predicted"})
)
df_res = pd.merge(df_res, df_max_fev, on=["ID"])
df_res["FEV delta"] = df_res["max ecFEV1 % Predicted"] - df_res["ecFEV1 % Predicted"]

In [223]:
# df_res.sort_values(by="FEV delta", ascending=False)[0:5]
df_res.sort_values(by="O2 Saturation (%)", ascending=False)[0:5]

KeyError: 'O2 Saturation (%)'

In [220]:
# i = 2955
# fev1 and o2: see impact of very low o2

# Shared hfev1/ar uncertainty: small healthy lungs or big lungs with disease
# unexpectedly big lungs with disease: high fev1 % pred delta (benefit of 2nd day)
indices = [18452, 33198, 2330, 2975, 18430]
# Unexpectedly small lungs: low fev1 % pred but no permanent lung damage (high fef2575%fev1) → show without fef2575 then with

indices = []

for i in indices:
    id = df.loc[i, "ID"]
    (
        _,
        inf_alg,
        HFEV1,
        uecFEV1,
        ecFEV1,
        AR,
        HO2Sat,
        O2SatFFA,
        IA,
        UO2Sat,
        O2Sat,
        ecFEF2575prctecFEV1,
    ) = mb.o2sat_fev1_fef2575_point_in_time_model_noise_shared_healthy_vars(
        df.iloc[i].Height,
        df.iloc[i].Age,
        df.iloc[i].Sex,
        ia_prior="uniform",
        ar_prior="uniform",
        ecfev1_noise_model_cpt_suffix="_std_add_mult_ecfev1",
        ar_fef2575_cpt_suffix="_ecfev1_2_days_model_add_mult_noise",
    )

    fig = make_subplots(
        rows=4,
        cols=3,
        # row_titles=[
        #     "Prior knowledge",
        #     "Impact of observing FEV1",
        #     "Impact of additionnally observing O2Sat",
        #     "Impact of adding a second day (with the max FEV1)",
        # ],
        shared_xaxes=True
    )
    first_data_col = 2


    def add_fev1_prct_pred_line(fig, val, y_max, row, col):
        fig.add_shape(
            type="line",
            # opacity=0.1,
            x0=val,
            y0=0,
            x1=val,
            y1=y_max * 1.1,
            line=dict(color="black", width=2, dash="dash"),
            # fillcolor="red",
            # line_width=0,
            row=row,
            col=col,
        )
        return -1


    def add_hists(fig, hfev1_col, ar_col, i, row):
        ih.plot_histogram(
            fig,
            HFEV1,
            df_res.loc[i, hfev1_col],
            HFEV1.a,
            HFEV1.b,
            row,
            first_data_col,
            f" {hfev1_col}",
            "#009e73",
        )
        ih.plot_histogram(
            fig,
            AR,
            df_res.loc[i, ar_col],
            AR.a,
            AR.b,
            row,
            first_data_col + 1,
            f" {ar_col}",
            "#d55e00",
            clean_ticks=True,
        )
        add_fev1_prct_pred_line(
            fig, df_res.iloc[0]["ecFEV1 % Predicted"], max(df_res.loc[i, ar_col]), row, first_data_col + 1
        )


    ih.plot_histogram(
        fig,
        HFEV1,
        HFEV1.cpt,
        HFEV1.a,
        HFEV1.b,
        1,
        first_data_col,
        HFEV1.name + " prior",
        "#009e73",
    )
    ih.plot_histogram(
        fig,
        AR,
        AR.cpt,
        AR.a,
        AR.b,
        1,
        first_data_col + 1,
        AR.name + " prior",
        "#d55e00",
        clean_ticks=True,
    )
    add_fev1_prct_pred_line(
        fig, df_res.iloc[0]["ecFEV1 % Predicted"], max(AR.cpt), 1, first_data_col + 1
    )

    add_hists(fig, "HFEV1_fev1", "AR_fev1", i, 2)
    add_hists(fig, "HFEV1_fev1_o2", "AR_fev1_o2", i, 3)
    add_hists(fig, "HFEV1_fev1_o2_2d", "AR_fev1_o2_2d", i, 4)

    # Add "prior knowledge" on col 1, row 1
    fig.add_annotation(
        x=0.2,
        y=3.5/4,
        text="Prior knowledge",
        xref="paper",
        yref="paper",
        xanchor="center",
        yanchor="middle",
        showarrow=False,
    )
    add_text_box(
        fig,
        text="Your text here<br>Second line<br>Third line",
        row=2,
        col=1
    )

    title = f"{id}, {df_res.loc[i, 'Date Recorded']}, i={i}"

    fig.update_layout(
        showlegend=False,
        height=500,
        width=800,
        font=dict(size=11),
        bargap=0.01,
        title=title
        # margin=dict(l=20, r=20, b=20, t=20),
    )

    # fig.show()
    fig.write_image(f"{dh.get_path_to_main()}PlotsBreathe/Narrowing_in/{title}.pdf")

(0.0, 0.2888888888888889) (0.5375, 0.73125)
(0.0, 0.2888888888888889) (0.5375, 0.73125)
(0.0, 0.2888888888888889) (0.5375, 0.73125)
(0.0, 0.2888888888888889) (0.5375, 0.73125)
(0.0, 0.2888888888888889) (0.5375, 0.73125)


In [170]:
def add_text_box(fig, text, row, col):
    # Get the subplot's domain coordinates
    x_domain = fig.get_subplot(row, col).xaxis.domain
    y_domain = fig.get_subplot(row, col).yaxis.domain
    print(x_domain, y_domain)

    # Add rectangle
    fig.add_shape(
        type="rect",
        x0=x_domain[0] - 1,
        x1=x_domain[1] * 25,
        y0=y_domain[0],
        y1=y_domain[1],
        line=dict(color="black", width=0),
        fillcolor="white",
        xref="paper",
        yref="paper",
        row=row,
        col=col,
    )

    # Add text
    fig.add_annotation(
        x=(x_domain[0] + x_domain[1]) / 2,  # center x
        y=(y_domain[0] + y_domain[1]) / 2,  # center y
        text=text,
        showarrow=False,
        font=dict(size=12),
        xref="paper",
        yref="paper",
        xanchor="center",
        yanchor="middle",
        row=row,
        col=col,
    )